# Metrics II Part 2 PS 3
Alternatively to the formulation from the lecture
$$A(\theta)X_t = B(\theta)\mathbb{E}\left[X_{t+1} \right] + C(\theta)X_{t-1} + D(\theta)\eta_t$$ with
$$X_t = \left[x_t \, \pi_t \, i_t \, g_t \, u_t\right]'$$ $$\eta_t = \left[\epsilon_{it} \, \epsilon_{gt} \, \epsilon_{ut} \right]'$$
I follow Chris Sims approach (see http://sims.princeton.edu/yftp/gensys/LINRE3A.pdf), which is coded up in FRBNY's DSGE package.
## (a)

In [1]:
using DSGE

σ = 1; κ = 0.2; β = 0.99; ϕ_π = 1.25; ϕ_x = 0.25; ρ_i = 0.5; 
σ_i = 2; ρ_g = 0.75; ρ_u = 0.75; σ_g = 1; σ_u = 1; σ_m = 0.1;

The equilibrium equations are:

$$x_t = \mathbb{E}_t x_{t+1} - \sigma(i_t-\mathbb{E}\pi_{t+1})+g_t$$
$$\pi_t = \kappa x_t+\beta \mathbb{E}_t \pi_{t+1}+u_t$$
$$i_t=\rho_i i_{t-1} + (1-\rho_i)(\phi_\pi \pi_t + \phi_x x_t) + \sigma_i \epsilon_{it}$$
$$g_t = \rho_g g_{t-1} + \sigma_g \epsilon_{gt}$$
$$u_t = \rho_u u_{t-1} + \sigma_u \epsilon_{ut}$$

Substituting expectations with outcomes plus errors, i.e. $x_{t+1}=\mathbb{E}_t \left[ x_{t+1} \right] + \eta_{t+1}$:

$$x_t = x_{t+1} - \eta_{xt+1} -\sigma(i_t -\pi_{t+1} +\eta_{\pi t+1})+g_t $$
$$\pi_t = \kappa x_t+\beta (\pi_{t+1} -\eta_{\pi t+1}) + u_t$$
$$i_t=\rho_i i_{t-1} + (1-\rho_i)(\phi_\pi \pi_t + \phi_x x_t) + \sigma_i \epsilon_{it}$$
$$g_t = \rho_g g_{t-1} + \sigma_g \epsilon_{gt}$$
$$u_t = \rho_u u_{t-1} + \sigma_u \epsilon_{ut}$$

The general form with expectational errors is:

$$\Gamma_0 \, y_t = \Gamma_1 \, y_{t-1} + c + \Psi z_t + \Pi \eta_t$$

I adopt the notation in which time arguments or subscripts
relate consistently to the information structure: variables dated t are always known at t.


Rearranging yields:

$$-x_{t} -\sigma \pi_{t} = -x_{t-1} -\sigma i_{t-1} +g_{t-1} -\eta_{xt} -\sigma \eta_{\pi t}$$
$$-\beta \pi_{t} = \kappa x_{t-1} -\pi_{t-1} + u_{t-1} -\beta \eta_{\pi t} $$
$$-(1-\rho_i) \phi_x x_t -(1-\rho_i) \phi_\pi \pi_t +i_t=\rho_i i_{t-1} +\sigma_i \epsilon_{it}$$
$$g_t = \rho_g g_{t-1} + \sigma_g \epsilon_{gt}$$
$$u_t = \rho_u u_{t-1} + \sigma_u \epsilon_{ut}$$

Define $y_t = \left[x_t \, \pi_t \, i_t \, g_t \, u_t\right]'$, $z_t = \left[\epsilon_{it} \, \epsilon_{gt} \, \epsilon_{ut} \right]'$ and $\eta_t = \left[\eta_{xt} \, \eta_{\pi t} \right]'$.

Now I am ready to write down the matrices:

In [2]:
Γ0 = [-1 -σ 0 0 0; 0 -β 0 0 0; -(1-ρ_i)ϕ_x -(1-ρ_i)ϕ_π 1 0 0; 0 0 0 1 0; 0 0 0 0 1]
Γ1 = [-1 0 -σ 1 0; κ -1 0 0 1; 0 0 ρ_i 0 0; 0 0 0 ρ_g 0; 0 0 0 0 ρ_u]
c = zeros(5)
Ψ = [0 0 0; 0 0 0; σ_i 0 0; 0 σ_g 0; 0 0 σ_u]
Π = [-1 -σ; 0 -β; 0 0; 0 0; 0 0];

## (b)

In [3]:
solution = gensys(Γ0, Γ1, c, Ψ, Π);
G1 = solution[1]

5×5 Array{Float64,2}:
  7.88142e-16  -2.2134e-16   -0.563271     1.18864      -1.09162    
 -1.01677e-16   4.53908e-16  -0.16626      0.591811      1.53767    
 -2.79052e-16   3.3819e-16    0.325679     0.518462      0.824592   
  1.34297e-16   9.92316e-17   3.24644e-16  0.75         -1.76301e-16
 -1.52211e-16   1.96528e-16  -1.80684e-16  1.77942e-16   0.75       

In [4]:
G2 = solution[3]

5×3 Array{Float64,2}:
 -2.25308      1.58486      -1.45549    
 -0.66504      0.789081      2.05023    
  1.30271      0.691283      1.09946    
  4.30612e-16  1.0          -1.49016e-16
 -1.62465e-16  2.96215e-17   1.0        

$$y_t = G_1 y_{t-1} + G_2 z_t$$

Observables:
$$Y_t = \left[x_t^{obs} \, \pi_t^{obs} \, i_t^{obs} \right]$$ with
$$Y_t = H(\theta)y_t + J(\theta)\nu_t$$ and:

In [5]:
H = [1 0 0 0 0; 0 1 0 0 0; 0 0 1 0 0]
J = eye(3) .* σ_m;

In [6]:
using Distributions
using Plots; gr()

N = 1000

y = zeros(5,N)
Y = zeros(3,N)

z = rand(MvNormal(zeros(3), eye(3)), N)
ν = rand(MvNormal(zeros(3), eye(3)), N)

for i in 2:size(y)[2]
    y[:,i] = G1*y[:,i-1] + G2*z[:,i]
    Y[:,i] = H*y[:,i] + J*ν[:,i]
end

In [7]:
plot(Y[1:3,:]', layout=3, title=["Output Gap" "Inflation" "Interest Rate"], legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 Output Gap 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,115.306 30.8738,110.937 31.144,130.889 31.4142,96.618 31.6844,100.525 31.9545,95.0215 32.2247,111.126 32.4949,106.815 32.7651,118.117 33.0352,121.914 
 33.3054,151.83 33.5756,119.935 33.8458,128.56 34.1159,99.111 34.3861,104.281 34.6563,126.006 34.9265,123.368 35.1966,111.652 35.4668,160.629 35.737,160.385 
 36.0072,149.663 36.2773,125.259 36.5475,127.642 36.8177,116.868 37.0878,98.4306 37.358,166.982 37.6282,137.977 37.8984,116.714 38.1685,142.288 38.4387,138.141 
 38.7089,137.838 38.9791,115.522 39.2492,103.717 39.5194,91.4247 39.7896,108.761 40.0598,105.102 40.3299,115.043 40.6001,107.978 40.8703,104.916 41.1405,133.083 
 41.4106,114.378 41.6808,116.872 41.951,84.2105 42.2212,104.321 42.4913,112.206 42.7615,102.793 43.0317,140.989 43.3018,104.081 43.572,118.613 43.8422,93.0783 
 44.1124,116.383 44.3825,119.506 44.6527,105.114 44.9229,138.374 45.1931,118.609 45.4632,117.685 45.7334,115.165 46.0036,131.196 46.2738,138.921 46.5439,145.313 
 46.8141,120.92 47.0843,115.897 47.3545,124.632 47.6246,141.007 47.8948,156.827 48.165,166.91 48.4352,129.245 48.7053,80.7079 48.9755,90.8785 49.2457,119.639 
 49.5159,128.7 49.786,148.643 50.0562,138.729 50.3264,143.497 50.5965,150.506 50.8667,155.047 51.1369,162.09 51.4071,133.009 51.6772,149.6 51.9474,132.999 
 52.2176,127.863 52.4878,107.833 52.7579,100.321 53.0281,125.134 53.2983,90.5097 53.5685,84.6038 53.8386,104.223 54.1088,109.936 54.379,94.4802 54.6492,140.16 
 54.9193,133.164 55.1895,153.137 55.4597,106.218 55.7299,102.069 56,80.6647 56.2702,91.3869 56.5404,124.077 56.8105,120.316 57.0807,133.356 57.3509,91.6473 
 57.6211,121.464 57.8912,152.339 58.1614,158.397 58.4316,155.815 58.7018,140.436 58.9719,165.873 59.2421,157.611 59.5123,177.347 59.7825,168.45 60.0526,165.7 
 60.3228,122.866 60.593,137.241 60.8632,104.64 61.1333,110.814 61.4035,89.5944 61.6737,118.989 61.9439,147.948 62.214,112.59 62.4842,135.594 62.7544,107.592 
 63.0245,134.829 63.2947,104.311 63.5649,138.208 63.8351,161.564 64.1052,139.557 64.3754,102.525 64.6456,111.333 64.9158,128.289 65.1859,100.012 65.4561,117.243 
 65.7263,131.487 65.9965,122.93 66.2666,113.241 66.5368,73.8177 66.807,121.275 67.0772,94.2196 67.3473,96.0256 67.6175,104.584 67.8877,113.748 68.1579,101.489 
 68.428,144.835 68.6982,161.019 68.9684,156.589 69.2385,144.24 69.5087,125.044 69.7789,114.264 70.0491,95.6155 70.3192,99.8776 70.5894,89.6561 70.8596,111.778 
 71.1298,168.889 71.3999,173.106 71.6701,149.864 71.9403,80.0832 72.2105,79.6607 72.4806,82.191 72.7508,62.0671 73.021,79.0554 73.2912,87.1747 73.5613,86.0903 
 73.8315,116.876 74.1017,148.317 74.3719,148.385 74.642,92.2207 74.9122,77.3861 75.1824,52.9469 75.4525,72.276 75.7227,76.9038 75.9929,103.521 76.2631,98.0502 
 76.5332,93.5319 76.8034,129.217 77.0736,121.806 77.3438,139.824 77.6139,148.103 77.8841,137.676 78.1543,111.146 78.4245,104.265 78.6946,91.9095 78.9648,145.549 
 79.235,134.373 79.5052,125.902 79.7753,93.919 80.0455,79.628 80.3157,55.1243 80.5859,64.9336 80.856,80.9128 81.1262,50.5793 81.3964,102.676 81.6665,96.9996 
 81.9367,78.7456 82.2069,53.0265 82.4771,80.8 82.7472,104.042 83.0174,94.6999 83.2876,95.652 83.5578,107.108 83.8279,123.093 84.0981,112.551 84.3683,124.065 
 84.6385,137.851 84.9086,139.05 85.1788,150.837 85.449,136.388 85.7192,131.445 85.9893,134.167 86.2595,131.909 86.5297,103.809 86.7999,76.3542 87.07,71.9348 
 87.3402,70.1456 87.6104,98.9103 87.8805,126.432 88.1507,136.098 88.4209,99.9225 88.6911,83.2292 88.9612,91.8718 89.2314,97.0953 89.5016,98.1504 89.7718,104.835 
 90.0419,130.546 90.3121,147.863 90.5823,118.764 90.8525,126.329 91.1226,101.929 91.3928,121.257 91.663,82.1036 91.9332,78.7475 92.2033,108.488 92.4735,142.711 
 92.7437,98.9547 93.0

## (d)
The likelihood function is:
$$p(Y^T|\theta) = p(Y_0|\theta) \prod_{t=1}^T p(Y_t|Y^{t-1}, \theta)$$

To calculate conditional likelihoods I define:
$$y_{t|t-1} = G_1 y_{t-1|t-1}$$
$$P_{t|t-1} = G_1 P_{t-1|t-1} G_1' + G_2 G_2'$$
$$Y_{t|t-1} = H y_{t|t-1}$$
$$V_{t|t-1} = H P_{t|t-1} H' + J J'$$
$$p_t = p_{t-1} - \frac{1}{2}\left( \log |V_{t|t-1}| + (Y_t - Y_{t|t-1}) V^{-1}_{t|t-1} (Y_t - Y_{t|t-1}) \right)$$
$$y_{t|t} = y_{t|t-1} + P_{t|t-1} H' V^{-1}_{t|t-1}(Y_t - Y_{t|t-1})$$
$$P_{t|t} = P_{t|t-1} - P_{t|t-1}' H' V^{-1}_{t|t-1} H P_{t|t-1}$$

In [8]:
y_forcast = zeros(5)
P_forcast = zeros(5, 5)
Y_forcast = zeros(3)
V_forcast = zeros(5, 5)

p = 0

y_nowcast = zeros(5)
P_nowcast = eye(5)

for i in 2:N-1
    y_forcast = G1 * y_nowcast
    P_forcast = G1 * P_nowcast * G1' + G2 * G2'
    Y_forcast = H * y_forcast
    V_forcast = H * P_forcast * H' + J * J'
    
    p += p - 1/2 * (log(det(V_forcast)) + (Y[:,i+1] - Y_forcast)' * inv(V_forcast) * (Y[:,i+1] - Y_forcast))[1]
    
    y_nowcast = y_forcast + P_forcast * H' * inv(V_forcast) * (Y[:,i+1] - Y_forcast)
    P_nowcast = P_forcast - P_forcast * H' * inv(V_forcast) * H * P_forcast
end

In [ ]:
det()

In [9]:
p

-1.0294426479286109e301

## (e)

In [10]:
function eval_likelihood(;σ = 1, κ = 0.2, β = 0.99, ϕ_π = 1.25, ϕ_x = 0.25, ρ_i = 0.5, 
                        σ_i = 2, ρ_g = 0.75, ρ_u = 0.75, σ_g = 1, σ_u = 1, σ_m = 0.1)
    
    Γ0 = [-1 -σ 0 0 0; 0 -β 0 0 0; -(1-ρ_i)ϕ_x -(1-ρ_i)ϕ_π 1 0 0; 0 0 0 1 0; 0 0 0 0 1]
    Γ1 = [-1 0 -σ 1 0; κ -1 0 0 1; 0 0 ρ_i 0 0; 0 0 0 ρ_g 0; 0 0 0 0 ρ_u]
    c = zeros(5)
    Ψ = [0 0 0; 0 0 0; σ_i 0 0; 0 σ_g 0; 0 0 σ_u]
    Π = [-1 -σ; 0 -β; 0 0; 0 0; 0 0]

    solution = gensys(Γ0, Γ1, c, Ψ, Π);
    G1 = solution[1]
    G2 = solution[3]
    H = [1 0 0 0 0; 0 1 0 0 0; 0 0 1 0 0]
    J = σ_m
    
    y_forcast = zeros(5)
    P_forcast = zeros(5, 5)
    Y_forcast = zeros(3)
    V_forcast = zeros(5, 5)
    
    p = 0
    
    y_nowcast = zeros(5)
    P_nowcast = eye(5)
    
    y_forcast_rec = zeros(5, N)
    
    for i in 2:N-1
        y_forcast = G1 * y_nowcast
        P_forcast = G1 * P_nowcast * G1' + G2 * G2'
        Y_forcast = H * y_forcast
        V_forcast = H * P_forcast * H' + J * J'
        
        p += p - 1/2 * (log(det(V_forcast)) + (Y[:,i+1] - Y_forcast)' * inv(V_forcast) * (Y[:,i+1] - Y_forcast))[1]
        
        y_nowcast = y_forcast + P_forcast * H' * inv(V_forcast) * (Y[:,i+1] - Y_forcast)
        P_nowcast = (eye(5) - P_forcast * H' * inv(V_forcast) * H) * P_forcast
    end
    p
end

eval_likelihood (generic function with 1 method)

In [11]:
plot(linspace(0.5,5,50), [eval_likelihood(σ=x)./1e300 for x in linspace(0.5,5,50)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -30 
 
 
 -20 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,4.82326 42.1437,4.0647 53.6836,3.93701 65.2236,4.36889 76.7636,5.30875 88.3036,6.71798 99.8436,8.56696 111.384,10.8324 122.924,13.4957 134.464,16.5415 
 146.004,19.9572 157.544,23.7321 169.084,27.857 180.623,32.3239 192.163,37.126 203.703,42.2568 215.243,47.7109 226.783,53.4831 238.323,59.5687 249.863,65.9633 
 261.403,72.6628 272.943,79.6632 284.483,86.961 296.023,94.5525 307.563,102.434 319.103,110.603 330.643,119.056 342.183,127.789 353.723,136.8 365.263,146.085 
 376.803,155.641 388.343,165.466 399.883,175.557 411.423,185.91 422.963,196.523 434.503,207.393 446.043,218.517 457.583,229.892 469.123,241.515 480.663,253.383 
 492.203,265.494 503.743,277.845 515.283,290.434 526.823,303.256 538.363,316.309 549.903,329.592 561.443,343.1 572.983,356.831 584.523,370.783 596.063,384.952 
 
 "/>
 
 
 
 
 y1

In [12]:
plot(linspace(0.0,1,50), [eval_likelihood(κ=x)./1e300 for x in linspace(0.0,1,50)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -16 
 
 
 -14 
 
 
 -12 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,49.7534 42.1437,39.8657 53.6836,31.6866 65.2236,24.9015 76.7636,19.2938 88.3036,14.7092 99.8436,11.0347 111.384,8.18544 122.924,6.09641 134.464,4.71718 
 146.004,4.00797 157.544,3.93701 169.084,4.47868 180.623,5.61212 192.163,7.3202 203.703,9.58874 215.243,12.4059 226.783,15.7619 238.323,19.6483 249.863,24.058 
 261.403,28.985 272.943,34.4241 284.483,40.3709 296.023,46.8215 307.563,53.7726 319.103,61.2212 330.643,69.1647 342.183,77.6009 353.723,86.5277 365.263,95.9432 
 376.803,105.846 388.343,116.234 399.883,127.106 411.423,138.462 422.963,150.299 434.503,162.616 446.043,175.413 457.583,188.688 469.123,202.441 480.663,216.67 
 492.203,231.375 503.743,246.554 515.283,262.206 526.823,278.332 538.363,294.928 549.903,311.996 561.443,329.533 572.983,347.538 584.523,366.012 596.063,384.952 
 
 "/>
 
 
 
 
 y1